In [11]:
import sys
import os
import gurobipy as gp
from gurobipy import GRB


trafficname = "cluster_c" # "unv1" # "prv1" # "cluster_c" # "cluster_b" # "cluster_a"
routingname = "su3" # "su2" # "su3" # "su3" # "su3" # "su3"
numinterval = 48 # 8 # 48
maxH = 8 # 4 # 8
scaledown = 10000000 # 100000 # 10000000
homedir = "/home/annzhou"
resultfile = f"{homedir}/DRing/src/emp/datacentre/experiments/temporal/{trafficname}.txt"


def compute(interval,graphfile,numsw,netpathfile,traffic,method,crossover,qivarfile):
    # read netpathfile
    netpath = list()
    for i in range(numsw):
        netpath.append(list())
        for j in range(numsw):
            netpath[i].append(list())
    with open(netpathfile,'r') as f:
        lines = f.readlines()
        # produce
        fromsw = 0
        tosw = 0
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
            else:
                path = [fromsw]
                tokens = line.split()
                for token in tokens:
                    hops = token.split("->")
                    path.append(int(hops[1]))
                netpath[fromsw][tosw].append(path)

        # check
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                numpaths = int(tokens[2])
                if len(netpath[fromsw][tosw])!=numpaths:
                    print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

    # read graphfile
    link = list()
    for i in range(numsw):
        link.append(list())
        for j in range(numsw):
            link[i].append(0)
    with open(graphfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split("->")
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            link[fromsw][tosw] = 1
            link[tosw][fromsw] = 1

    # read linkfailurefile (if needed)
    # if numfaillink > 0:
    #     with open(linkfailurefile,'r') as f:
    #         lines = f.readlines()
    #         for line in lines:
    #             tokens = line.split()
    #             fromsw = int(tokens[0])
    #             tosw = int(tokens[1])
    #             if link[fromsw][tosw] != 1:
    #                 print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
    #             else:
    #                 link[fromsw][tosw] /= 2

    # read serverfile
    # serverdict = dict()
    # with open(serverfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(',')
    #         serverdict[int(tokens[0])] = int(tokens[1])

    # read flowfile
    # traffic = list()
    # for i in range(numsw):
    #     traffic.append(list())
    #     for j in range(numsw):
    #         traffic[i].append(0)
    # with open(flowfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(",")
    #         fromsvr = int(tokens[0])
    #         tosvr = int(tokens[1])
    #         if fromsvr>=numserver or tosvr>=numserver: continue
    #         fromsw = serverdict[fromsvr]
    #         tosw = serverdict[tosvr]
    #         if fromsw == tosw: continue
    #         flowbytes = int(tokens[2])
    #         traffic[fromsw][tosw] += flowbytes


    # precompute
    flowsvialink = list()
    for i in range(numsw):
        flowsvialink.append(list())
        for j in range(numsw):
            flowsvialink[i].append(list())
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                for pid,path in enumerate(netpath[fromsw][tosw]):
                    fidpidstr = f"{fromsw},{tosw},{pid}"
                    prevhop = fromsw
                    for hop in path[1:]:
                        flowsvialink[prevhop][hop].append(fidpidstr)
                        prevhop = hop

    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         traffic[interval][fromsw][tosw] /= 100000


    # Create a new model
    model = gp.Model("mcf")

    # Add variables
    maxpid = 0
    for fromsw in range(numsw):
        for tosw in range(numsw):
            maxpid = max(maxpid,len(netpath[fromsw][tosw]))
    vararr = list()
    for fromsw in range(numsw):
        vararr.append(list())
        for tosw in range(numsw):
            vararr[fromsw].append(list())
            for pid in range(maxpid):
                vararr[fromsw][tosw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                    vararr[fromsw][tosw][pid] = var
    k = model.addVar(name="k")

    # Set objective function
    model.setObjective(k, GRB.MAXIMIZE)

    # Add constraints
    # Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[interval][fromsw][tosw] > 0:
                varlist = list()
                for pid in range(len(netpath[fromsw][tosw])):
                    varlist.append(vararr[fromsw][tosw][pid])
                model.addConstr(sum(varlist)>=k*traffic[interval][fromsw][tosw],f"c0_{fromsw}_{tosw}")

    # Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
    for linkfrom in range(numsw):
        for linkto in range(numsw):
            flowstrlist = flowsvialink[linkfrom][linkto]
            if len(flowstrlist) > 0:
                varlist = list()
                for flowstr in flowstrlist:
                    tokens = flowstr.split(',')
                    flowfrom = int(tokens[0])
                    flowto = int(tokens[1])
                    pid = int(tokens[2])
                    varlist.append(vararr[flowfrom][flowto][pid])
                model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

    # Constraint 2: for each fid: for each pid: pid = prev_pid
    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         if traffic[interval][fromsw][tosw] > 0:
    #             for pid in range(1,len(netpath[fromsw][tosw])):
    #                 model.addConstr(vararr[fromsw][tosw][pid]-vararr[fromsw][tosw][pid-1]==0,f"c2_{fromsw}_{tosw}_{pid}")

    # Optimize model
    model.setParam('Method',method)
    model.setParam('Crossover',crossover)
    model.optimize()
    # model.write(modelfile)

    # Print results
    if model.status == GRB.OPTIMAL:
        print(f"Optimal objective value: {model.objVal}")
        with open(qivarfile,'a') as f:
            f.write(f"{routingname},{interval},{k.x}\n")
            for fromsw in range(numsw):
                for tosw in range(numsw):
                    if traffic[interval][fromsw][tosw] > 0:
                        for pid in range(len(netpath[fromsw][tosw])):
                            var = vararr[fromsw][tosw][pid]
                            f.write(f"{var.varName},{var.x}\n")
    else:
        print("No optimal solution found")
    return


def solve(computeinterval,computeqivarfile,solveinterval,graphfile,numsw,netpathfile,traffic,cachefile,method,crossover):
    if os.path.exists(cachefile):
        with open(cachefile,'r') as f:
            lines = f.readlines()
            for line in lines:
                tokens = line.split(',')
                mycomputeinterval = int(tokens[0])
                mysolveinterval = int(tokens[1])
                mythroughput = float(tokens[2])
                if computeinterval==mycomputeinterval and solveinterval==mysolveinterval:
                    return mythroughput

    if not os.path.exists(computeqivarfile):
        compute(computeinterval,graphfile,numsw,netpathfile,traffic,method,crossover,computeqivarfile)


    # read netpathfile
    netpath = list()
    for i in range(numsw):
        netpath.append(list())
        for j in range(numsw):
            netpath[i].append(list())
    with open(netpathfile,'r') as f:
        lines = f.readlines()
        # produce
        fromsw = 0
        tosw = 0
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
            else:
                path = [fromsw]
                tokens = line.split()
                for token in tokens:
                    hops = token.split("->")
                    path.append(int(hops[1]))
                netpath[fromsw][tosw].append(path)

        # check
        for line in lines:
            if "->" not in line:
                tokens = line.split()
                fromsw = int(tokens[0])
                tosw = int(tokens[1])
                numpaths = int(tokens[2])
                if len(netpath[fromsw][tosw])!=numpaths:
                    print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

    # read graphfile
    link = list()
    for i in range(numsw):
        link.append(list())
        for j in range(numsw):
            link[i].append(0)
    with open(graphfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split("->")
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            link[fromsw][tosw] = 1
            link[tosw][fromsw] = 1

    # read linkfailurefile (if needed)
    # if numfaillink > 0:
    #     with open(linkfailurefile,'r') as f:
    #         lines = f.readlines()
    #         for line in lines:
    #             tokens = line.split()
    #             fromsw = int(tokens[0])
    #             tosw = int(tokens[1])
    #             if link[fromsw][tosw] != 1:
    #                 print(f"ERROR: should have a link from {fromsw} to {tosw} but not")
    #             else:
    #                 link[fromsw][tosw] /= 2

    # read serverfile
    # serverdict = dict()
    # with open(serverfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(',')
    #         serverdict[int(tokens[0])] = int(tokens[1])

    # read flowfile
    # traffic = list()
    # for i in range(numsw):
    #     traffic.append(list())
    #     for j in range(numsw):
    #         traffic[i].append(0)
    # with open(flowfile,'r') as f:
    #     lines = f.readlines()
    #     for line in lines:
    #         tokens = line.split(",")
    #         fromsvr = int(tokens[0])
    #         tosvr = int(tokens[1])
    #         if fromsvr>=numserver or tosvr>=numserver: continue
    #         fromsw = serverdict[fromsvr]
    #         tosw = serverdict[tosvr]
    #         if fromsw == tosw: continue
    #         flowbytes = int(tokens[2])
    #         traffic[fromsw][tosw] += flowbytes

    # read computeqivarfile
    pathweight = list()
    for i in range(numsw):
        pathweight.append(list())
        for j in range(numsw):
            pathweight[i].append(dict())
    with open(computeqivarfile,'r') as f:
        lines = f.readlines()[1:]
        for line in lines:
            tokens = line.split(",")
            string = tokens[0]
            stringtokens = string.split("_")
            fromsw = int(stringtokens[0])
            tosw = int(stringtokens[1])
            pid = int(stringtokens[2])
            weight = float(tokens[1])
            pathweight[fromsw][tosw][pid] = weight
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw]>0 and traffic[computeinterval][fromsw][tosw]==0:
                numnetpath = len(netpath[fromsw][tosw])
                for i in range(numnetpath):
                    pathweight[fromsw][tosw][i] = 1.0/numnetpath


    # precompute
    flowsvialink = list()
    for i in range(numsw):
        flowsvialink.append(list())
        for j in range(numsw):
            flowsvialink[i].append(list())
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid,path in enumerate(netpath[fromsw][tosw]):
                    fidpidstr = f"{fromsw},{tosw},{pid}"
                    prevhop = fromsw
                    for hop in path[1:]:
                        flowsvialink[prevhop][hop].append(fidpidstr)
                        prevhop = hop

    # for fromsw in range(numsw):
    #     for tosw in range(numsw):
    #         traffic[interval][fromsw][tosw] /= 100000


    # Create a new model
    model = gp.Model("mcf")

    # Add variables
    maxpid = 0
    for fromsw in range(numsw):
        for tosw in range(numsw):
            maxpid = max(maxpid,len(netpath[fromsw][tosw]))
    vararr = list()
    for fromsw in range(numsw):
        vararr.append(list())
        for tosw in range(numsw):
            vararr[fromsw].append(list())
            for pid in range(maxpid):
                vararr[fromsw][tosw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    var = model.addVar(name=f"{fromsw}_{tosw}_{pid}")
                    vararr[fromsw][tosw][pid] = var
    k = model.addVar(name="k")
    avararr = list()
    for fromsw in range(numsw):
        avararr.append(list())
        for tosw in range(numsw):
            avararr[fromsw].append(None)
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                avar = model.addVar(name=f"a_{fromsw}_{tosw}")
                avararr[fromsw][tosw] = avar

    # Set objective function
    model.setObjective(k, GRB.MAXIMIZE)

    # Add constraints
    # Constraint 0: for each fid: sum(pid) >= k * traffic[fid.from][fid.to]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                varlist = list()
                for pid in range(len(netpath[fromsw][tosw])):
                    varlist.append(vararr[fromsw][tosw][pid])
                model.addConstr(sum(varlist)>=k*traffic[solveinterval][fromsw][tosw],f"c0_{fromsw}_{tosw}")

    # Constraint 1: for each link: sum(fid_pid) <= link[link.from][link.to]
    for linkfrom in range(numsw):
        for linkto in range(numsw):
            flowstrlist = flowsvialink[linkfrom][linkto]
            if len(flowstrlist) > 0:
                varlist = list()
                for flowstr in flowstrlist:
                    tokens = flowstr.split(',')
                    flowfrom = int(tokens[0])
                    flowto = int(tokens[1])
                    pid = int(tokens[2])
                    varlist.append(vararr[flowfrom][flowto][pid])
                model.addConstr(sum(varlist)<=link[linkfrom][linkto],f"c1_{linkfrom}_{linkto}")

    # Constraint 2: for each fid: for each pid: pid = pathweight[fid][pid] * a[fid]
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if traffic[solveinterval][fromsw][tosw] > 0:
                for pid in range(len(netpath[fromsw][tosw])):
                    model.addConstr(vararr[fromsw][tosw][pid]==pathweight[fromsw][tosw][pid]*avararr[fromsw][tosw],f"c2_{fromsw}_{tosw}_{pid}")

    # Optimize model
    model.setParam('Method',-1)
    model.setParam('Crossover',0)
    model.optimize()
    # model.write(modelfile)

    # Print results
    if model.status == GRB.OPTIMAL:
        print(f"Optimal objective value: {model.objVal}")
        with open(cachefile,'a') as f:
            f.write(f"{computeinterval},{solveinterval},{k.x}\n")
            # for fromsw in range(numsw):
            #     for tosw in range(numsw):
            #         if traffic[fromsw][tosw] > 0:
            #             for pid in range(len(netpath[fromsw][tosw])):
            #                 var = vararr[fromsw][tosw][pid]
            #                 f.write(f"{var.varName},{var.x}\n")
    else:
        print("No optimal solution found")
    return k.x


def find_max_indices(numbers, tolerance):
    # Find the maximum value in the list
    max_value = max(numbers)
    
    # Find all indices where the value is within the tolerance of the maximum value
    indices = [i for i, value in enumerate(numbers) if abs(value - max_value)/max_value <= tolerance]
    
    return indices


def write_arr(file,arr,prefix,suffix):
    file.write(prefix)
    for a in arr:
        file.write(f"{a},")
    file.write(suffix)


# set up parameters
if trafficname == "cluster_b":
    trafficfilesuffixlist = ["0_500","500_1000","1000_1500","1500_2000","2000_2500","2500_2900"]
else:
    trafficfilesuffixlist = ["0_273"]
step_s = 1800
numserver = 2988
numsw = 80
numport = 64
# numinterval = 86400//step_s
graphname = "dring"
numfaillink = 0
numlink = 2132
fseed = 0
method = 2
crossover = 0
factor = 64
error_tolerance = 0.01
routingnamearr = ["ecmp","su2","su3","32disjoint"]

if graphname=="dring":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/double_ring/instance1_{numsw}_{numport}.edgelist"
elif graphname=="rrg":
    graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/rrg/instance1_{numsw}_{numport}.edgelist"
else:
    print("ERROR: leafspine does not need to compute path weight")

# spatialfile = f"{homedir}/DRing/src/emp/datacentre/computerouting2/output/spatial_{graphname}_{trafficname}_-1_0_{factor}_d{scaledown}"
# modelfile = f"{homedir}/DRing/src/emp/datacentre/makepathweightfiles/model.lp"
# resultfile = f"{homedir}/DRing/src/emp/datacentre/computerouting2/resultfiles/result_{graphname}_{numfaillink}_{fseed}_{trafficname}_{method}_{crossover}"
# processfile = f"{homedir}/DRing/src/emp/datacentre/computerouting2/processfiles/process_{graphname}_{numfaillink}_{fseed}_{trafficname}_{method}_{crossover}"


# read in traffic
serverfile = f"{homedir}/DRing/src/emp/datacentre/serverfiles/dring_{numserver}_{numsw}_{numport}"
serverdict = dict()
with open(serverfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        serverdict[int(tokens[0])] = int(tokens[1])

traffic = list()
for k in range(numinterval):
    traffic.append(list())
    for i in range(numsw):
        traffic[k].append(list())
        for j in range(numsw):
            traffic[k][i].append(0)
if trafficname.startswith("cluster_"):
    for suffix in trafficfilesuffixlist:
        trafficfile = f"{homedir}/DRing/src/emp/datacentre/trafficfiles/{trafficname}/traffic_64racks_{suffix}"
        with open(trafficfile,'r') as f:
            lines = f.readlines()
            for line in lines:
                tokens = line.split()
                time_s = int(tokens[0])
                interval = time_s//step_s
                fromsvr = int(tokens[1])
                tosvr = int(tokens[2])
                if fromsvr>=numserver or tosvr>=numserver: continue
                fromsw = serverdict[fromsvr]
                tosw = serverdict[tosvr]
                if fromsw == tosw: continue
                flowbytes = int(tokens[3])
                traffic[interval][fromsw][tosw] += flowbytes
else:
    trafficfile = f"{homedir}/DRing/src/emp/datacentre/trafficfiles/{trafficname}"
    with open(trafficfile,'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.split(',')
            interval = int(tokens[0])
            fromsvr = int(tokens[1])
            tosvr = int(tokens[2])
            if fromsvr>=numserver or tosvr>=numserver: continue
            fromsw = serverdict[fromsvr]
            tosw = serverdict[tosvr]
            if fromsw == tosw: continue
            flowbytes = int(tokens[3])
            traffic[interval][fromsw][tosw] += flowbytes
for k in range(numinterval):
    for i in range(numsw):
        for j in range(numsw):
            traffic[k][i][j] /= scaledown

totaltraffic_per_interval = list()
for k in range(numinterval):
    totaltraffic = 0
    for i in range(numsw):
        for j in range(numsw):
            totaltraffic += traffic[k][i][j]
    totaltraffic_per_interval.append(totaltraffic)


# read in throughput with equal path weights
# equalthroughput = dict()
# for routingname in routingnamearr:
#     equalthroughput[routingname] = dict()
# with open(spatialfile,'r') as f:
#     lines = f.readlines()
#     for line in lines:
#         tokens = line.split(',')
#         routingname = tokens[0]
#         interval = int(tokens[1])
#         throughput = float(tokens[2])
#         equalthroughput[routingname][interval] = throughput


# compute & solve
with open(resultfile,'a') as fprocess:
    for H in range(maxH+1):
        for solveinterval in range(numinterval//2,numinterval):
            computeinterval = solveinterval-H
            computeqivarfile = f"{homedir}/DRing/src/emp/datacentre/qivarfiles/ivar_{graphname}_{numfaillink}_{fseed}_{routingname}_{trafficname}_{computeinterval}_{method}_{crossover}"
            cachefile = f"{homedir}/DRing/src/emp/datacentre/computerouting2/cachefiles/cache_{graphname}_{numfaillink}_{fseed}_{routingname}_{trafficname}_{method}_{crossover}"
            netpathfile = f"{homedir}/DRing/src/emp/datacentre/netpathfiles/netpath_{routingname}_{graphname}.txt"
            mythroughput = solve(computeinterval,computeqivarfile,solveinterval,graphfile,numsw,netpathfile,traffic,cachefile,method,crossover)
            fprocess.write(f"{solveinterval},{computeinterval},{mythroughput}\n")

Set parameter Method to value 2
Set parameter Crossover to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) Silver 4110 CPU @ 2.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 5298 rows, 783000 columns and 3104159 nonzeros
Model fingerprint: 0x87e48a26
Coefficient statistics:
  Matrix range     [5e-07, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 1.84s
Presolved: 5298 rows, 783000 columns, 3104159 nonzeros
Ordering time: 0.20s

Barrier statistics:
 Dense cols : 1
 AA' NZ     : 1.570e+06
 Factor NZ  : 3.156e+06 (roughly 340 MB of memory)
 Factor Ops : 3.485e+09 (less than 1 second per iteration)
 Threads    : 16

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0 